# Biological sequence analysis

Here we have all the known proteome from E.coli. It's all nicely kept in a fasta file "ecoli_proteome.fasta" in this directory. Let's open the file on the directory menu on your left and see how it looks

This file has a collection of entries, which have two parts: 
- Header: Always starts with `>` and contains information about the sequence. For istance, the UniprotID, given name, genome location... any information relavant to identify the protein.
- Sequence: The list of aminoacids... or nucleotides! Because the fasta format is also used to store dna or rna sequences!

Let's read the first entry in the high-level python way:

In [ ]:
f = open('ecoli_proteome.fasta', 'r')

read = True

# Not the easiest logic. But this is a way of reading the first sequence
for line in f: 
    if line.startswith('>') and read:
        print(line.rstrip())
        read=False
    elif line.startswith('>') and not read:
        break
    else:
        print(line.rstrip())
        
f.close()

What if we want the second sequence only? 

In [ ]:
f = open('ecoli_proteome.fasta', 'r')

entry_count = 0
target_sequence = 2


for line in f: 
    if line.startswith('>'):
        entry_count += 1
        if entry_count == target_sequence:
            print(line.rstrip())
    elif entry_count == target_sequence:
        print(line.rstrip())
    
        
f.close()

What if you want multiple sequences?

In [ ]:
f = open('ecoli_proteome.fasta', 'r')

entry_count = 0
target_sequence = [2,5,10,42]


for line in f: 
    if line.startswith('>'):
        entry_count += 1
        if entry_count in target_sequence:
            print(line.rstrip())
    elif entry_count in target_sequence:
        print(line.rstrip())
    
        
f.close()

**It's not that it's impossible** to do things this way, but the inconvenience is major and I hate it with all my soul... Plus, my sequences are unindexed for downstream analysis. **There is a better way**. 

## Reading Fastas with Biopython

Biopython is a package that allows to do many of the analysis that we need on biology and bioinformatics. It's not installed by default on anaconda, but if you run the cell bellow it should start with the installation. **Please don't run it if you have already installed it**. 

In [ ]:
!conda install -c conda-forge biopython -y

Now, we can read fastas in a more efficient way, and it keeps it all indexed. This means that I can select the sequences that I want using their index, or iterate with a for loop over them, or do anything that you can do with indexed objects. This is the way to do it: 

In [ ]:
# First we import the necessary package from biopython
from Bio import SeqIO

In [ ]:
my_fasta_entries = []
for record in SeqIO.parse('ecoli_proteome.fasta', 'fasta'):
    my_fasta_entries.append(record)

You can find more documentation about SeqIO at https://biopython.org/wiki/SeqIO

Now let's look at the first entry:

In [ ]:
print(my_fasta_entries[0])

In [ ]:
print(type(my_fasta_entries[0]))

So what is this? 

This is an **object** that contains the **properties** id, name, seq and other properties. How do we know this?

In [ ]:
print(my_fasta_entries[0].__doc__)  # Print documentation of the object. This shows the name of the attributes and their description.
                                    # This will only be available if the developer has taken their time to write the documentation.

In [ ]:
my_fasta_entries[0].__dict__    # We can see all the properties of the object here, but without the explanation of what they are.
                                # This will always be available, regardless on whether someone took the time to write documentation.

So we can select the different properties of the object using a dot followed by the name of the property. Keep in mind that `my_fasta_entries` is a **list** of **objects**, so we first need to select an object from the list to be able to select its attribute. Or said in code:

In [ ]:
print(type(my_fasta_entries))
print(type(my_fasta_entries[0]))
print(type(my_fasta_entries[42]))

In [ ]:
# Works
print(my_fasta_entries[0].id)
print(my_fasta_entries[0].name)
print(my_fasta_entries[0].description)
print(my_fasta_entries[0].seq)

In [ ]:
# Doesn't work
print(my_fasta_entries.id)

Just as if it was a chain of events, we can operate the output of each operation normally. 

Below, we do the following things:
- We select the object in position 0 of our list of objects
- Then we select only the id **property** of the object, which is a string
- Following, we split the selected string using the character `|` as the delimiter, which results in a list. 
- Finally, we select the element in position 1 of that list.

You can see it nicely chopped here. 

In [ ]:
print('Select object and print it')
print(my_fasta_entries[0])

print('\nSelect object and print its id property')
print(my_fasta_entries[0].id)

print('\nSelect object, then its id and split it when it finds the character |')
print(my_fasta_entries[0].id.split('|'))

print('\nSelect object, then its id, split it when it finds the character | and select the element in position one of the splitted list')
print(my_fasta_entries[0].id.split('|')[1])  # This is the UniprotID

### Homework 1

Create a dictionary that contains the UniprotID of all proteins as `keys` and their sequence as `values`. Then, print the dictionary entry with the key `'P10089'`.

## Sequence Alignment with Biopython

FIRST OF ALL, you can find more formal explanation of the following part here: https://biopython.org/DIST/docs/api/Bio.pairwise2-module.html

In [ ]:
from Bio import pairwise2

In [ ]:
alignments = pairwise2.align.globalxx("ACCGT", "ACG")  #this performs a global alignment

In [ ]:
print(alignments) # Ugly, what is this?

So we see 2 alignments here. This means that there are 2 alignments that share the maximum alignment score. **A higher score means that the two sequences are closer to each other than two sequences of the same length with a lower score**. There are ways of obtaining normalized scores, but we won't go into this in this tutorial. 

In [ ]:
print(alignments[0])
print(alignments[1])

Equally ugly, but now we have an alignment at a time. If we were looking only at the score we would select it from the alignment, but where is it? 

In [ ]:
print(pairwise2.align.globalxx.__doc__)

So our according to our documentation our score is in position 2. Therefore the scores of the alignments are:

In [ ]:
print(alignments[0][2])
print(alignments[1][2])

But wait, we still haven't fixed how ugly it looks. The score might be interesting for large scale analysis, but I only have 2 proteins that I want to know about, I want to see **how** they align... Don't worry fam:

In [ ]:
print(pairwise2.format_alignment(*alignments[0]))  
print(pairwise2.format_alignment(*alignments[1]))  

# Don't worry about the asterisk, it means to take "everything" from the variable (alignments[0]). 
# It has all the information that alignments [0] contains, but not in a tuple form anymore. This is merely an implementation issue

### Changing gap and missmatch penalties.

So far, our alignments have been done using an identity matrix. This means that it scored 1 if two positions matched and 0 if they mismatched. The gap penalty was also set to 0, which means that the alignment algorithm can insert gaps "for free". 

This doesn't make sense from a biological point of view, since a gap or a missmatch indicates a mutation, which should correspond to a negative score (because this means the sequences are less similar).

Let's start assigning both gap and missmatch a penalty of 1 and to match a score of 2:

Notice that there is an xx in pairwise2.align.globalxx? This has a meaning. The first letter defines the match score, and the second the gap score. 

The match parameters are:

CODE  DESCRIPTION
- x     No parameters. Identical characters have score of 1, otherwise 0.
- m     A match score is the score of identical chars, otherwise mismatch
      score.
- d     A dictionary returns the score of any pair of characters.
- c     A callback function returns scores.

The gap penalty parameters are:

CODE  DESCRIPTION
- x     No gap penalties.
- s     Same open and extend gap penalties for both sequences.
- d     The sequences have different open and extend gap penalties.
- c     A callback function returns the gap penalties.

Therefore if we we do same alignments with first the definition of match and missmatch it would look like this: 

In [ ]:
# Custom match and missmatch scores
alignments = pairwise2.align.globalmx("ACCGT", "ACG", 2,-1)  #this performs a global alignment
print(pairwise2.format_alignment(*alignments[0]))
print(pairwise2.format_alignment(*alignments[1]))

In [ ]:
# Custom gap opening and gap extension penalties
alignments = pairwise2.align.globalxs("ACCGT", "ACG",-2,-1)  #this performs a global alignment
print(pairwise2.format_alignment(*alignments[0]))
print(pairwise2.format_alignment(*alignments[1]))

In [ ]:
# Custom all
alignments = pairwise2.align.globalms("ACCGT", "ACG", 2,-1, -2, -1)  #this performs a global alignment
print(pairwise2.format_alignment(*alignments[0]))
print(pairwise2.format_alignment(*alignments[1]))

### Proteins and substitution matrices 

Proteins are made of amino acids. Amino acids have physical properties that make some of them be closer to each other. That's why a substitution from a Leucine to an Isoleucine shouldn't be scored the same as a chenge from a Gylcine to an Proline. To fix this, the so called **Substitution Matrices** were calculated. 

Historically, the PAM matrices were used. They were calculated by hand and simulated evolution during a milion years at a time. The most famous was the **PAM250**, which simulated evolution during 250 years. 

Nowadays, new matrices have been calculated, based on multiple sequence alignment with certain identity threshold, the Blosum matrices. The most famous is **Blosum62**, which is derived from a multiple sequence alignment of sequences with at least 62% identity. 

These matrices illustrate how deleterious a substitution can be depending on which amino acid was substituted and for which one. **Keep in mind that these matrices do not include any kind of gap penalties**. 

Here's how Blosum62 looks like:

<!-- ![title](BLOSUM62.png) -->

<figure>
   
   <img src="BLOSUM62.png">
   <figcaption> Blosum62 substitution matrix. Source: Wikipedia.
   </figcaption>
</figure>

**Let's align two proteins with Blosum62**

In [ ]:
from Bio.SubsMat import MatrixInfo as matlist
matrix = matlist.blosum62  # Import blossum62
print(type(matrix))  # This means that we need to change the first 'x' for a 'd' in the globalxx() function

In [ ]:
seq_1 = my_fasta_entries[0].seq
seq_2 = my_fasta_entries[1].seq

In [ ]:
alignments = pairwise2.align.globalds(seq_1, seq_2, matrix, -2, -1)  #this performs a global alignment
print(pairwise2.format_alignment(*alignments[0]))  # and it doesn't look pretty for long sequences

In [ ]:
print('Alignment score is {0}'.format(alignments[0][2]))

### Homework 2

Perform a global alignment of the **A)** first sequence in our E.coli fasta against **B)** every protein in the fasta file (including itself). Use the matrix Blosum62 and gap penalties of -2 and -0.8 for opening and extension respectively. 

Store the scores as values of a dictionary that takes the UniprotID of the second protein (**B**) as keys. This is the structure of the dictionary, but with 500+ entries:

my_beloved_dictionary = {'P04743':-42, 'P08663': 69}

# GOOD LUCK!